<a href="https://colab.research.google.com/github/Diggi14/project_Property2/blob/main/Data_filling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Diggi14/project_Property2.git

Cloning into 'project_Property2'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 25 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 1.12 MiB | 3.77 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
df=pd.read_csv('/content/project_Property2/gurgaon_cleaned.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9396 entries, 0 to 9395
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           9396 non-null   int64  
 1   PREFERENCE           9396 non-null   object 
 2   PROPERTY_TYPE        9396 non-null   object 
 3   BEDROOM_NUM          9392 non-null   float64
 4   BATHROOM_NUM         9396 non-null   int64  
 5   BALCONY_NUM          9382 non-null   float64
 6   PRICE_PER_UNIT_AREA  9396 non-null   float64
 7   FURNISH              9103 non-null   object 
 8   AGE                  9394 non-null   object 
 9   FLOOR_NUM            9396 non-null   float64
 10  TOTAL_FLOOR          9396 non-null   float64
 11  SUPERBUILTUP_SQFT    9396 non-null   float64
 12  PRICE                9396 non-null   float64
 13  LOCALITY_WO_CITY     9396 non-null   object 
 14  stories              9396 non-null   object 
dtypes: float64(7), int64(2), object(6)
mem

In [6]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [7]:
df.head(4)

,PREFERENCE,PROPERTY_TYPE,BEDROOM_NUM,BATHROOM_NUM,BALCONY_NUM,PRICE_PER_UNIT_AREA,FURNISH,AGE,FLOOR_NUM,TOTAL_FLOOR,SUPERBUILTUP_SQFT,PRICE,LOCALITY_WO_CITY,stories
0,S,Residential Apartment,4.0,4,4.0,8766.0,Semi-furnished,5-10,14.0,26.0,3434.0,2.63,Sector 84,single
1,S,Residential Apartment,4.0,4,3.0,21176.0,Semi-furnished,1-5,7.0,29.0,2870.0,3.60,Sector 81,single
2,S,Residential Apartment,3.0,3,3.0,13740.0,Semi-furnished,1-5,14.0,28.0,2802.0,3.85,Sector 112,single
3,S,Residential Apartment,3.0,4,4.0,8515.0,Semi-furnished,1-5,4.0,27.0,2290.0,1.95,Sector 104,single


In [8]:
x=df.drop('PRICE',axis=1)
y=df['PRICE']

In [9]:
num_cols = ['BEDROOM_NUM','BATHROOM_NUM','PRICE_PER_UNIT_AREA','FLOOR_NUM','TOTAL_FLOOR','SUPERBUILTUP_SQFT']
cat_cols = ['PREFERENCE','PROPERTY_TYPE','FURNISH','AGE','LOCALITY_WO_CITY','stories','BALCONY_NUM']

num_pipeline = Pipeline([
    ('imputer', KNNImputer()),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

param_grid = {
    'preprocessor__num__imputer__n_neighbors': [2, 3, 5, 7, 10]
}

grid = GridSearchCV(pipeline, param_grid, cv=5)
grid.fit(x, y)

print("Best n_neighbors:", grid.best_params_)

Best n_neighbors: {'preprocessor__num__imputer__n_neighbors': 10}


In [10]:
df_numeric = df.select_dtypes(include=['number'])

imputer = KNNImputer(n_neighbors=10)
df_numeric = pd.DataFrame(
    imputer.fit_transform(df_numeric),
    columns=df_numeric.columns
)

df[df_numeric.columns] = df_numeric


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9396 entries, 0 to 9395
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PREFERENCE           9396 non-null   object 
 1   PROPERTY_TYPE        9396 non-null   object 
 2   BEDROOM_NUM          9396 non-null   float64
 3   BATHROOM_NUM         9396 non-null   float64
 4   BALCONY_NUM          9396 non-null   float64
 5   PRICE_PER_UNIT_AREA  9396 non-null   float64
 6   FURNISH              9103 non-null   object 
 7   AGE                  9394 non-null   object 
 8   FLOOR_NUM            9396 non-null   float64
 9   TOTAL_FLOOR          9396 non-null   float64
 10  SUPERBUILTUP_SQFT    9396 non-null   float64
 11  PRICE                9396 non-null   float64
 12  LOCALITY_WO_CITY     9396 non-null   object 
 13  stories              9396 non-null   object 
dtypes: float64(8), object(6)
memory usage: 1.0+ MB


In [12]:
df['FURNISH'] = df['FURNISH'].fillna(df['FURNISH'].mode()[0])

In [13]:
imputer = SimpleImputer(strategy='most_frequent')

df[['AGE']] = imputer.fit_transform(df[['AGE']])

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9396 entries, 0 to 9395
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PREFERENCE           9396 non-null   object 
 1   PROPERTY_TYPE        9396 non-null   object 
 2   BEDROOM_NUM          9396 non-null   float64
 3   BATHROOM_NUM         9396 non-null   float64
 4   BALCONY_NUM          9396 non-null   float64
 5   PRICE_PER_UNIT_AREA  9396 non-null   float64
 6   FURNISH              9396 non-null   object 
 7   AGE                  9396 non-null   object 
 8   FLOOR_NUM            9396 non-null   float64
 9   TOTAL_FLOOR          9396 non-null   float64
 10  SUPERBUILTUP_SQFT    9396 non-null   float64
 11  PRICE                9396 non-null   float64
 12  LOCALITY_WO_CITY     9396 non-null   object 
 13  stories              9396 non-null   object 
dtypes: float64(8), object(6)
memory usage: 1.0+ MB


In [16]:
df['LOCALITY_WO_CITY'].value_counts()

,count
LOCALITY_WO_CITY,
Sector 65,310
Sector 102,259
Sector 48,241
Sohna,226
DLF Phase 2,214
Sector-33 Sohna,205
Sector 92,198
Sector 54,190
Sector 67,188


In [17]:
df[df['LOCALITY_WO_CITY'].str.contains('block')]

,PREFERENCE,PROPERTY_TYPE,BEDROOM_NUM,BATHROOM_NUM,BALCONY_NUM,PRICE_PER_UNIT_AREA,FURNISH,AGE,FLOOR_NUM,TOTAL_FLOOR,SUPERBUILTUP_SQFT,PRICE,LOCALITY_WO_CITY,stories
8275,S,Independent/Builder Floor,10.0,10.0,4.0,8438.0,Furnished,0-1,5.0,5.0,2785.0,2.35,C block Sector 43,single
8279,S,Independent/Builder Floor,10.0,10.0,4.0,7259.0,Furnished,0-1,5.0,5.0,2700.0,1.96,C block Sector 43,single
8280,S,Independent/Builder Floor,10.0,10.0,4.0,6666.0,Furnished,0-1,5.0,5.0,2700.0,1.80,C block Sector 43,single


In [18]:
df.loc[df[df['LOCALITY_WO_CITY'].str.contains('block')].index,'LOCALITY_WO_CITY']=df.loc[df[df['LOCALITY_WO_CITY'].str.contains('block')].index,'LOCALITY_WO_CITY'].str.split('block').str.get(1).str.strip()

In [19]:
df[df['LOCALITY_WO_CITY'].str.startswith('Block')].index

Index([  21,  230,  231, 1146, 1600, 1978, 2125, 2146, 2500, 2771, 2772, 3140,
       3803, 4156, 4641, 4700, 4742, 5134, 5288, 5756, 5986, 6543, 6811, 6877,
       6878, 6955, 7401, 7441, 7527, 7606, 7610, 7719, 7752, 8475],
      dtype='int64')

In [21]:
ind = df[df['LOCALITY_WO_CITY'].str.startswith('Block')].index

df.loc[ind, 'LOCALITY_WO_CITY']=df.loc[ind, 'LOCALITY_WO_CITY'].str.replace(r'^Block\s+\w+\s*', '', regex=True)

In [31]:
ind = df[df['LOCALITY_WO_CITY'].str.startswith('Block')].index
df.loc[ind, 'LOCALITY_WO_CITY']=df.loc[ind, 'LOCALITY_WO_CITY'].str.split(',').str.get(1)

In [37]:
ind=df[df['LOCALITY_WO_CITY'].str.contains('Block')].index
df.loc[ind,'LOCALITY_WO_CITY']=df.loc[ind,'LOCALITY_WO_CITY'].str.split('Block').str.get(1).str.strip()

In [45]:
ind=df[df['LOCALITY_WO_CITY'].str.contains('Sushant Lok Phase - 3')].index

In [48]:
df.loc[ind,'LOCALITY_WO_CITY']=df.loc[ind,'LOCALITY_WO_CITY'].str.replace(' -','')

In [49]:
df['LOCALITY_WO_CITY'].value_counts()

,count
LOCALITY_WO_CITY,
Sector 65,310
Sector 102,259
Sector 48,241
Sushant Lok Phase 1,233
Sohna,226
DLF Phase 2,214
Sector-33 Sohna,205
Sector 92,198
Sector 54,190


In [55]:
ind=df[df['LOCALITY_WO_CITY'].str.contains('Manesar')].index
df.loc[ind,'LOCALITY_WO_CITY']=df.loc[ind,'LOCALITY_WO_CITY'].str.split('I').str.get(0)

In [58]:
ind=df[df['LOCALITY_WO_CITY'].str.contains(', Sushant Lok Phase - 2')].index
df.loc[ind,'LOCALITY_WO_CITY']=df.loc[ind,'LOCALITY_WO_CITY'].str.replace(',','').str.replace(' -','')

In [63]:
df.loc[df[df['LOCALITY_WO_CITY'].str.contains('Pocket I')].index,'LOCALITY_WO_CITY']=df.loc[df[df['LOCALITY_WO_CITY'].str.contains('Pocket I')].index,'LOCALITY_WO_CITY'].str.replace('Pocket I ','')

In [69]:
df.loc[df[df['LOCALITY_WO_CITY'].str.contains('DLF phase 2')].index,'LOCALITY_WO_CITY']=df.loc[df[df['LOCALITY_WO_CITY'].str.contains('DLF phase 2')].index,'LOCALITY_WO_CITY'].str.replace('DLF phase 2','DLF Phase 2')

In [73]:
df.loc[df[df['LOCALITY_WO_CITY'].str.contains(', Mayfield garden')].index,'LOCALITY_WO_CITY']=df.loc[df[df['LOCALITY_WO_CITY'].str.contains(', Mayfield garden')].index,'LOCALITY_WO_CITY'].str.replace(',','').str.strip()

In [74]:
df.loc[df[df['LOCALITY_WO_CITY'].str.contains(', Mayfield garden')].index,'LOCALITY_WO_CITY']=df.loc[df[df['LOCALITY_WO_CITY'].str.contains('DLF City Phase 1')].index,'LOCALITY_WO_CITY'].str.replace('DLF City Phase 1','DLF Phase 1').str.strip()

In [75]:
df.loc[df[df['LOCALITY_WO_CITY'].str.contains('Sector 9A Sector 9')].index,'LOCALITY_WO_CITY']=df.loc[df[df['LOCALITY_WO_CITY'].str.contains('Sector 9A Sector 9')].index,'LOCALITY_WO_CITY'].str.replace('Sector 9A Sector 9','Sector 9').str.strip()
df.loc[df[df['LOCALITY_WO_CITY'].str.contains('sector 47 , sector 51')].index,'LOCALITY_WO_CITY']=df.loc[df[df['LOCALITY_WO_CITY'].str.contains('sector 47 , sector 51')].index,'LOCALITY_WO_CITY'].str.replace('sector 47 , sector 51','Sector 47').str.strip()

In [79]:
df.drop(4654,inplace=True)

In [86]:
ind=df[df['LOCALITY_WO_CITY'].str.contains('vatika floor')].index
df.loc[ind,'LOCALITY_WO_CITY']=df.loc[ind,'LOCALITY_WO_CITY'].str.replace('vatika floor','')

In [91]:
ind=df[df['LOCALITY_WO_CITY'].str.contains('sector 28 , Golf course road')].index
df.loc[ind,'LOCALITY_WO_CITY']=df.loc[ind,'LOCALITY_WO_CITY'].str.split(',').str.get(0).str.strip()

In [94]:
ind=df[df['LOCALITY_WO_CITY'].str.contains('Sector 2 Palam Vihar')].index
df.loc[ind,'LOCALITY_WO_CITY']=df.loc[ind,'LOCALITY_WO_CITY'].str.replace('Sector 2 Palam Vihar','Sector 2')

In [97]:
ind=df[df['LOCALITY_WO_CITY'].str.contains('DLF City Phase 1')].index
df.loc[ind,'LOCALITY_WO_CITY']=df.loc[ind,'LOCALITY_WO_CITY'].str.replace('DLF City Phase 1','DLF Phase 1')

In [98]:
ind=df[df['LOCALITY_WO_CITY'].str.contains('Sec 90')].index
df.loc[ind,'LOCALITY_WO_CITY']=df.loc[ind,'LOCALITY_WO_CITY'].str.replace('Sec 90','Sector 90')

In [105]:
ind=df[df['LOCALITY_WO_CITY'].str.contains('Valley View Estate gwal Pahari')].index
df.loc[ind,'LOCALITY_WO_CITY']=df.loc[ind,'LOCALITY_WO_CITY'].str.split('e').str.get(-1)

In [108]:
ind=df[df['LOCALITY_WO_CITY'].str.contains('Sector 83 New')].index
df.loc[ind,'LOCALITY_WO_CITY']=df.loc[ind,'LOCALITY_WO_CITY'].str.replace('Sector 83 New','Sector 83')

In [111]:
ind=df[df['LOCALITY_WO_CITY'].str.contains('Sector - 33, Sohna,')].index
df.loc[ind,'LOCALITY_WO_CITY']=df.loc[ind,'LOCALITY_WO_CITY'].str.replace('Sector - 33, Sohna,','Sector-33 Sohna')

In [124]:
ind=df[df['LOCALITY_WO_CITY'].str.contains('gwal Pahari')].index
df.loc[ind,'LOCALITY_WO_CITY']=df.loc[ind,'LOCALITY_WO_CITY'].str.replace('g','G')

In [138]:
ind=df[df['LOCALITY_WO_CITY'].str.contains('oasis')].index
df.loc[ind,'LOCALITY_WO_CITY']=df.loc[ind,'LOCALITY_WO_CITY'].str.replace('emaar uarban oasis','')

In [ ]:
ind=df[df['LOCALITY_WO_CITY'].str.contains('Rosewood City')].index

In [139]:
df['LOCALITY_WO_CITY']=df['LOCALITY_WO_CITY'].str.strip()

In [140]:
df['LOCALITY_WO_CITY'].value_counts()

,count
LOCALITY_WO_CITY,
Sector 65,310
Sector 102,259
Sector 48,241
Sushant Lok Phase 1,233
Sohna,226
DLF Phase 2,217
Sector-33 Sohna,206
Sector 92,198
Sector 54,190


In [144]:
ind=df[df['LOCALITY_WO_CITY'].str.contains('110')].index
df.loc[ind,'LOCALITY_WO_CITY'].value_counts()

,count
LOCALITY_WO_CITY,
Sector 110,37
Sector 110 A,11


In [145]:
df.to_csv('cleaned_data2.csv', index=False)